In [48]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy import linalg
import algoritmo_ncm as ncm
import sys
import os

In [49]:

class HiddenPrints:
    """Classe que desabilita prints dentro de funções"""
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [50]:
def gera_matrizes(n):
    #Devolve uma matriz inicial e a que queremos achar a ncm
    x = np.random.randn(n, n)
    g = np.random.randn(n, n)

    #A matriz a qual queremos encontrar a ncm mais proxima precisa ser simétrica 
    g = g@g.T
    return x,g 

In [51]:
def teste_solução(x,g,max_iter,tol):
#Função testa se a solução é valida
    x, iter = ncm.sm_ncm(x,g,max_iter,tol)
    if np.any(np.linalg.eigvals(x)) < -tol:
        print("não resolve")
    else: 
        print("resolve")

In [52]:
def media_iter(n,laps,max_iter,tol):
    lista_iter = []
    for i in range(laps):
        x,g  = gera_matrizes(n)
        with HiddenPrints():
            x, iter = ncm.sm_ncm(x,g,max_iter,tol)
        lista_iter.append(iter)
    media = np.mean(np.array(lista_iter))
    return lista_iter, media


In [64]:
# Gerar as matrizes fora do bloco timeit
n = 30
laps = 100
lista_g = []
lista_x0 = []
for i in range(laps):
    x0, g = gera_matrizes(n)
    lista_g.append(g)
    lista_x0.append(x0)

def mede_tempo(lista_x0,lista_g,laps,max_iter,tol):
    """Uma função que mede o tempo sem gerar as matrizes nem os vetores,o resultado é o tempo
    de 100 iterações"""
    with HiddenPrints():
        for i in range(laps):
            x, iter = ncm.sm_ncm(lista_x0[i],lista_g[i],max_iter,0.00001)
    return x

%timeit mede_tempo(lista_x0,lista_g,100,2000,0.00001)

13 s ± 64.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
import numpy as np
from func_auxiliares_ncm import *


def atualiza_diag_x(x, g):
    """Função responsável por fazer uma iteração do algoritmo"""
    df = np.linalg.inv(np.diag(np.diag(derivada_clarke(x))))@(e(x) - np.diag(derivada_clarke(x)@sem_diag_g(g)))
    return df


def nearest_correlation_matrix(x_input, g_input, iter = 1000, tol = 10**(-6)):
    """Algoritmo de Nearest Correlation Matrix"""
    iteracao = 1
    X_anterior = x_input
    X_atual = np.diag(atualiza_diag_x(X_anterior, g_input)) + sem_diag_g(g_input)

    while iteracao < iter:
        iteracao = iteracao + 1
        print("iteracao", iteracao - 1)
        print(derivada_clarke(X_anterior) @ X_atual)
        X_anterior = X_atual
        X_atual = np.diag(atualiza_diag_x(X_anterior, g_input)) + sem_diag_g(g_input)

       # if np.linalg.norm(np.diag(derivada_clarke(X_atual) @ X_atual) - e(X_atual)) < tol:
       #     break
        if np.linalg.norm(derivada_clarke(X_atual) - derivada_clarke(X_anterior)) < tol:
            break      
        
    return derivada_clarke(X_anterior) @ X_atual, iteracao

In [29]:
import numpy as np

def projecao_S(R):

    eigvals, eigvecs = np.linalg.eigh(R)
    eigvals[eigvals < 0] = 0
    return eigvecs @ np.diag(eigvals) @ eigvecs.T

def projecao_U(Y):

    np.fill_diagonal(Y, 1)
    return Y

def projecoes_alternadas(G, max_iteracoes=1000):

    delta_S = np.zeros_like(G)
    X = G.copy()

    
    
    for k in range(1, max_iteracoes + 1):
        print(k)
        X_anterior = X.copy()

        R = X - delta_S
        
        # Projeta no subconjunto semidefinido
        Y = projecao_S(R)
        
        delta_S = Y - R
        
        # Projeta no subconjunto de diagonal unitária
        X = projecao_U(Y)
        
        # Critério de parada
        if np.linalg.norm(X - X_anterior) / np.linalg.norm(X) < 1e-6:
            break
        
        X_anterior = X.copy()
    return X



In [ ]:
#crie uma matriz G 4x4 e teste com as funções projeções alternadas e sm_ncm
G = np.random.randn(4, 4)
G = G @ G.T
X = projecoes_alternadas(G)
print(X)
X, iter = nearest_correlation_matrix(X, G, 1000, 1e-6)
print(X)




In [30]:
new_matrix = np.array([
    [2, -1, 0, 0],
    [-1, 2, -1, 0],
    [0, -1, 2, -1],
    [0, 0, -1, 2]
])

In [31]:
# Teste da função projeções alternadas com new_matrix
X_new = projecoes_alternadas(new_matrix)
print("Resultado da projeção alternada:")
print(X_new)

# Teste da função sm_ncm com new_matrix
X_sm_ncm, iter_sm_ncm = nearest_correlation_matrix(X_new, new_matrix, 1000, 1e-6)
print("Resultado do sm_ncm:")
print(X_sm_ncm)
print("Número de iterações:", iter_sm_ncm)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
Resultado da projeção alternada:
[[ 1.         -0.80841271  0.19158729  0.106775  ]
 [-0.80841271  1.         -0.65623329  0.19158729]
 [ 0.19158729 -0.65623329  1.         -0.80841271]
 [ 0.106775    0.19158729 -0.80841271  1.        ]]
iteracao 1
[[ 1.         -0.8084125   0.1915875   0.10677505]
 [-0.80841249  1.         -0.6562327   0.19158751]
 [ 0.19158751 -0.6562327   1.         -0.80841249]
 [ 0.10677505  0.1915875  -0.8084125   1.        ]]
Resultado do sm_ncm:
[[ 1.         -0.8084125   0.1915875   0.10677505]
 [-0.8084125   1.         -0.65623269  0.1915875 ]
 [ 0.1915875  -0.65623269  1.         -0.8084125 ]
 [ 0.10677505  0.1915875  -0.8084125   1.        ]]
Número de iterações: 2
